In [ ]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime

In [ ]:
db = "test.db"

query_sessions = '''
SELECT id, notes FROM session
'''

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:int)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", str(name))[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")
    
with closing(sqlite3.connect(db)) as conn:
    sessions = pd.read_sql(query_sessions, conn)
    sessions["date"] = sessions["id"].apply(get_date)


    print(sessions)